In [1]:
#!/bin/env python

from boxes import *
from learner import *
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import sys
sys.path.append("..")
from my_boxes_learner import *

/home/zonghaiyao/anaconda3/envs/run_boxmodel/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
with open('../data/ontologies/anatomy/alignment_data.pkl', 'rb') as f:
    all_alignment_data = pickle.load(f)
hm_align_score = pd.read_csv('../data/ontologies/anatomy/human_mouse_alignment_score.tsv', index_col=0)
hm_align_score

,2738,2739,2742,2744,2743,2745,2747,2746,2749,2750,...,6027,6028,6029,6030,6031,6032,6033,6034,5809,6035
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.258668
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.116256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2734,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2735,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2736,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [4]:
PATH = '../data/ontologies/anatomy/'

use_wandb = False
ats = 0.8                       # aligment training split
Transitive_Closure = False      # Transitive closure
box_type_dict = {'MinMaxSigmoidBoxes': MinMaxSigmoidBoxes,
                 'SigmoidBoxes': SigmoidBoxes,
                 'DeltaBoxes': DeltaBoxes,
                 'MinMaxBoxes': MinMaxBoxes}


class Config():
    def __init__(self):
        self.dims = 8
        self.lr = 0.0536756989641208
        self.rns_ratio = 1
        self.box_type = 'MinMaxSigmoidBoxes'
        self.pos_margin = 0.801342772182741
        self.neg_margin = 0.29315808430739754
        
config = Config()

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

print("is use cuda?", use_cuda)

is use cuda? True


In [6]:
if Transitive_Closure:
    tc = "tc_"
else:
    tc = ""

# Data in unary.tsv are probabilites separated by newlines. The probability on line n is P(n), where n is the id assigned to the nth element.
unary_prob = torch.from_numpy(np.loadtxt(f'{PATH}unary/unary.tsv')).float().to(device)
num_boxes = unary_prob.shape[0]

# We're going to use random negative sampling during training, so no need to include negatives in our training data itself
train = Probs.load_from_julia(PATH, f'tr_pos_{tc}{ats}.tsv', f'tr_neg_{ats}.tsv', ratio_neg = 0).to(device)

# The dev set will have a fixed set of negatives, however.
dev = Probs.load_from_julia(PATH, f'dev_align_pos_{ats}.tsv', f'dev_align_neg_{ats}.tsv', ratio_neg = 1).to(device)

# This set is used just for evaluation purposes after training
tr_align = Probs.load_from_julia(PATH, f'tr_align_pos_{ats}.tsv', f'tr_align_neg_{ats}.tsv', ratio_neg = 1).to(device)

mouse_eval = Probs.load_from_julia(PATH, 'human_dev_pos.tsv', 'human_dev_neg.tsv', ratio_neg = 1).to(device)
human_eval = Probs.load_from_julia(PATH, 'mouse_dev_pos.tsv', 'mouse_dev_neg.tsv', ratio_neg = 1).to(device)

In [8]:
#debugging
def metric_hard_accuracy_align(model, data_in, data_out, threshold:float):
    #here
    A_given_B = data_in[::2]
    B_given_A = data_in[1::2,:]
    data_out = data_out[::2]
    
#     print(data_in.shape, data_out.shape)
#     print(A_given_B)
#     print(B_given_A)
#     print(data_in)
#     print(data_out)

    align_probs = torch.stack((model(A_given_B)["P(A|B)"], model(B_given_A)["P(A|B)"]), dim=1)
    p = torch.min(align_probs, dim=1).values
    hard_pred = p > threshold
    
#     print("metric_hard_accuracy_align:", (data_out == hard_pred).float().mean())

    return (data_out == hard_pred).float().mean()

def metric_hard_f1_align(model, data_in, data_out, threshold:float):
    A_given_B = data_in[::2]
    B_given_A = data_in[1::2,:]
    data_out = data_out[::2]

    align_probs = torch.stack((model(A_given_B)["P(A|B)"], model(B_given_A)["P(A|B)"]), dim=1)
    p = torch.min(align_probs, dim=1).values
    hard_pred = p > threshold

    true_pos = data_out[hard_pred==1].sum()
    total_pred_pos = (hard_pred==1).sum().float()
    total_actual_pos = data_out.sum().float()

    precision = true_pos / total_pred_pos
    recall = true_pos / total_actual_pos
    
    #print("metric_hard_f1_align:", 2 * (precision*recall) / (precision + recall))

    return 2 * (precision*recall) / (precision + recall)

In [13]:
#implementing
# @dataclass
# class TrueNegativeSampling(Callback):
#     """
#     Given a pair (u,v) generates random pairs of the form (u,x) or (y,v)
#     """
#     num_entities: int
#     ratio: int
    
#     def batch_begin(self, l: Learner):
#         with torch.no_grad():
# #             print("batch_in before rns:", l.batch_in)
# #             print("batch_out before rns:", l.batch_out)

#             batch_in = l.batch_in.to("cpu") # Should be a tensor of indices, shape (batch_size, k)            
#             batch_size, k = batch_in.shape
#             num_neg_samples = batch_in.shape[0] * self.ratio
#             negative_samples = batch_in.repeat(self.ratio, 1) # shape (batch_size * ratio, k)
#             negative_probs = torch.zeros(num_neg_samples).to(l.batch_out.device)
#             #negative_samples.scatter_(1, torch.randint(k,(num_neg_samples,1)), torch.randint(self.num_entities, (num_neg_samples,1)))  
#             for i in range(batch_size):
#                 if self.split_data(l.batch_in[i], 2737) == 0:#all mouse
                    
#                 if self.split_data(l.batch_in[i], 2737) == 1:#all human
                    
#                 if self.split_data(l.batch_in[i], 2737) == 2:#human first
                    
#                 else:                                        #mouse first
                    
            
#             generate_true_neg_alignments(all_alignment_data, )
            
#             negative_samples = negative_samples.to(l.batch_in.device)
            
#             #for max_margin to re-construct the data_in and data_out
#             negative_samples = torch.reshape(negative_samples, (batch_size, self.ratio, k))
#             negative_probs = torch.reshape(negative_probs, (batch_size, self.ratio))

#             l.batch_in = torch.cat((l.batch_in.unsqueeze(1), negative_samples), 1)
#             l.batch_out = torch.cat((l.batch_out.unsqueeze(-1), negative_probs), -1)

            
#     def get_siblings(self, parents:dict, children:dict, node:int):
#         siblings = []

#         # There should only be only one node that doesn't have any parents, the root node
#         if node in parents:
#             parents_of_node = parents[node]

#             # Cycle through all possible parents of the given node
#             for p in parents_of_node:

#                 # if the parent node has any children, add them to the siblings list
#                 if p in children:
#                     siblings = siblings + children[p]

#                     # remove the node from the siblings list
#                     siblings.remove(node)

#             # if there are any siblings, return the list of them
#             if siblings:
#                 return siblings

#             # if there are no siblings, return -1
#             else:
#                 print("Given node does not have any siblings:", node)
#                 return -1

#         # if the node does not have any parents, return -1
#         else:
#             print("Given node does not have any parents:", node)
#             return -1
#         # ---- 

#     def generate_true_neg_alignments(self, alignments:list, alignment_split:float=0.5, ratio:float=1.0):

#         true_negatives = []
#         numFailures = 0
#         num_samples = int(len(alignments) * alignment_split * ratio)

#         while (len(true_negatives) < num_samples) and (numFailures < 100):
#             # Select a random alignment within the list of all alignments
#             rdm_align = random.choice(alignments)

#             # Pick a node to alter within the randomly chosen alignment 
#             const_node = rdm_align[0]
#             change_node = rdm_align[1]

#             # generate all siblings within the human ontology of the chosen node
#             if change_node in h_parents:
#                 siblings = get_siblings(h_parents, h_children, change_node)

#             # generate all siblings within the mouse ontology of the chosen node
#             elif change_node in m_parents:
#                 siblings = get_siblings(m_parents, m_children, change_node)

#             # This shouldn't be triggered -- every node should have a parent node
#             # The only possible node that could trigger the below statement is the root node
#             else:
#                 print("Node not found in either Ontology or does not have any parents")


#             # This error will typically be thrown if the chosen node does not have any siblings
#             if siblings == -1:
#                 print("Error thrown when retrieving siblings")

#             else:
#                 # Choose some random siblings to be make the true negative
#                 negative_alignment = (const_node, random.choice(siblings))

#                 if negative_alignment in alignments:
#                     numFailures += 1
#                     print("Generated negative is an existing alignment:", negative_alignment, "OG random:", rdm_align, siblings)
#                     pass

#                 elif negative_alignment in true_negatives:
#                     numFailures += 1
#                     print("Generated negative already in true_negatives:", negative_alignment)
#                     pass

#                 # include this negative alignment in the true_negatives list
#                 else:
#                     true_negatives.append(negative_alignment)
#                     true_negatives.append((negative_alignment[1], const_node))
#                     numFailures = 0


#         return true_negatives
    
#     def split_data(self, data, split):
#         a, b = data > split
        
#         #check if it is an alignment data
#         if not a and not b: #all mouse
#             return 0
#         elif a and b:       #all human
#             return 1
#         elif a:
#             return 2        #human first
#         else:
#             return 3        #mouse first

In [16]:
box_model = BoxModel(
    BoxParamType=box_type_dict[config.box_type],
    vol_func=soft_volume,
    num_models=1,
    num_boxes=num_boxes,
    dims=config.dims,
    method="orig").to(device)

#### IF YOU ARE LOADING FROM JULIA WITH ratio_neg=0, train_dl WILL ONLY CONTAIN POSITIVE EXAMPLES
#### THIS MEANS YOUR MODEL SHOULD USE NEGATIVE SAMPLING DURING TRAINING
train_dl = TensorDataLoader(train, batch_size=2**6, shuffle=True)

mouse_dl = TensorDataLoader(mouse_eval, batch_size=2**6)
human_dl = TensorDataLoader(human_eval, batch_size=2**6)

eval_dl = [mouse_dl, human_dl]

opt = torch.optim.Adam(box_model.parameters(), lr=config.lr)


# loss_func = LossPieces( (config.mouse_human_eps,mouse_cond_kl_loss), (config.mouse_human_eps,human_cond_kl_loss), (config.alignment_eps,align_cond_kl_loss))
loss_func = LossPieces(mouse_cond_margin_loss, human_cond_margin_loss, align_cond_margin_loss)

metrics = [metric_hard_accuracy, metric_hard_f1]
align_metrics = [metric_hard_accuracy_align, metric_hard_f1_align, metric_hard_accuracy_align_mean, metric_hard_f1_align_mean]

rec_col = RecorderCollection()

callbacks = CallbackCollection(
    LossCallback(rec_col.train, train),
    LossCallback(rec_col.dev, dev),
    *(MetricCallback(rec_col.dev, dev, "dev", m, use_wandb) for m in metrics),
    *(MetricCallback(rec_col.train, train, "train", m, use_wandb) for m in metrics),
    *(MetricCallback(rec_col.onto, human_eval, "human", m, use_wandb) for m in metrics),
    *(MetricCallback(rec_col.onto, mouse_eval, "mouse", m, use_wandb) for m in metrics),
    *(EvalAlignment(rec_col.train_align, tr_align, "train_align", m, use_wandb) for m in align_metrics),
    *(EvalAlignment(rec_col.dev_align, dev, "dev_align", m, use_wandb) for m in align_metrics),
    JustGiveMeTheData(rec_col.probs, dev, get_probabilities),
    BiasMetric(rec_col.bias, dev, pct_of_align_cond_on_human_as_min),
    PlotMetrics(rec_col.dev_roc_plot, dev, roc_plot),
    PlotMetrics(rec_col.dev_pr_plot, dev, pr_plot),
    PlotMetrics(rec_col.tr_roc_plot, tr_align, roc_plot),
    PlotMetrics(rec_col.tr_pr_plot, tr_align, pr_plot),
    MetricCallback(rec_col.train, train, 'train', metric_pearson_r, use_wandb),
    MetricCallback(rec_col.train, train, 'train', metric_spearman_r, use_wandb),
    MetricCallback(rec_col.dev, dev, 'dev', metric_pearson_r, use_wandb),
    MetricCallback(rec_col.dev, dev, 'dev', metric_spearman_r, use_wandb),
#     MetricCallback(rec_col.train, train, 'train', mean_reciprocal_rank, use_wandb),
#     MetricCallback(rec_col.dev, dev, 'dev', mean_reciprocal_rank, use_wandb),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mean_cond_kl_loss", 0.25, 10),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mean_cond_kl_loss", 0.5),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mouse_cond_kl_loss", 0.25, 10),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mouse_cond_kl_loss", 0.5),
#     GradientClipping(-1000,1000),
    #RandomNegativeSampling(num_boxes, config.rns_ratio),
    RandomNegativeSamplingWithAMLScore(num_boxes, config.rns_ratio),
    StopIfNaN(),
)

# l = Learner(train_dl, box_model, loss_func, opt, callbacks, recorder = rec_col.learn)
l = Learner(train_dl, box_model, loss_func, opt, callbacks, recorder = rec_col.learn, categories=True, use_wandb = use_wandb)

In [17]:
nEpochs = 1

In [18]:
#the log of init loss
l.train(nEpochs)

evaluation_dev_metric_hard_accuracy tensor(0.4975, device='cuda:0')
evaluation_dev_metric_hard_f1 tensor(0.0287, device='cuda:0')
evaluation_train_metric_hard_accuracy tensor(0.0214, device='cuda:0')
evaluation_train_metric_hard_f1 tensor(0.0420, device='cuda:0')
evaluation_human_metric_hard_accuracy tensor(0.5029, device='cuda:0')
evaluation_human_metric_hard_f1 tensor(0.0430, device='cuda:0')
evaluation_mouse_metric_hard_accuracy_1 tensor(0.5017, device='cuda:0')
evaluation_mouse_metric_hard_f1_1 tensor(0.0429, device='cuda:0')
evaluation_train_metric_pearson_r tensor(0., device='cuda:0')
evaluation_train_metric_spearman_r nan
evaluation_dev_metric_pearson_r tensor(-0.0035, device='cuda:0')
evaluation_dev_metric_spearman_r -0.009932850380536125


/home/zonghaiyao/anaconda3/envs/run_boxmodel/lib/python3.7/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/zonghaiyao/anaconda3/envs/run_boxmodel/lib/python3.7/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.5210, device='cuda:0')
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.0836, device='cuda:0')
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.5804, device='cuda:0')
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.2891, device='cuda:0')
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5000, device='cuda:0')
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(nan, device='cuda:0')
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.5000, device='cuda:0')
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.0257, device='cuda:0')
evaluation_dev_metric_hard_accuracy tensor(0.4967, device='cuda:0')
evaluation_dev_metric_hard_f1 tensor(0.0785, device='cuda:0')
evaluation_train_metric_hard_accuracy tensor(0.3786, device='cuda:0')
evaluation_train_metric_hard_f1 tensor(0.5492, device='cuda:0')
evaluation_huma

In [15]:
#the log of init loss
l.train(nEpochs)

evaluation_dev_metric_hard_accuracy tensor(0.5000, device='cuda:0')
evaluation_dev_metric_hard_f1 tensor(0.0350, device='cuda:0')
evaluation_train_metric_hard_accuracy tensor(0.0119, device='cuda:0')
evaluation_train_metric_hard_f1 tensor(0.0236, device='cuda:0')
evaluation_human_metric_hard_accuracy tensor(0.4984, device='cuda:0')
evaluation_human_metric_hard_f1 tensor(0.0281, device='cuda:0')
evaluation_mouse_metric_hard_accuracy_1 tensor(0.5004, device='cuda:0')
evaluation_mouse_metric_hard_f1_1 tensor(0.0325, device='cuda:0')
evaluation_train_metric_pearson_r tensor(0., device='cuda:0')
evaluation_train_metric_spearman_r nan
evaluation_dev_metric_pearson_r tensor(-0.0043, device='cuda:0')
evaluation_dev_metric_spearman_r 0.001740371220521287


/home/zonghaiyao/anaconda3/envs/run_boxmodel/lib/python3.7/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/zonghaiyao/anaconda3/envs/run_boxmodel/lib/python3.7/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5375, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.5367, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5041, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.5754, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5215, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5118, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.5215, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5881, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.5326, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.2939, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5396, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5453, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.5474, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5128, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.5733, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5396, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5097, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.5000, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5577, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.5350, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.3286, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5322, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5651, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.5802, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5231, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.5893, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5330, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5368, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4917, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5600, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.5614, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.3961, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5499, device='cuda:0')
a

...

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5758, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.6530, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5482, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6480, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5033, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5790, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4967, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5960, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.6022, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.6040, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5845, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5692, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.6483, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5412, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6415, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5066, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5830, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4917, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5958, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.5989, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.5991, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5829, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5705, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.6499, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5445, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6439, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5116, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5866, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4934, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5955, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.5965, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.5986, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5796, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5668, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.6462, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5400, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6407, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5033, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5814, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4950, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5963, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.5874, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.5903, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5758, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.5754, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.6539, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5412, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6429, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.5165, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5914, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4983, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5957, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.5973, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.6014, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.5787, device='cuda:0')
a

In [17]:
#the log of init loss
l.train(nEpochs)

evaluation_dev_metric_hard_accuracy tensor(0.5000, device='cuda:0')
evaluation_dev_metric_hard_f1 tensor(0.0901, device='cuda:0')
evaluation_train_metric_hard_accuracy tensor(0.6756, device='cuda:0')
evaluation_train_metric_hard_f1 tensor(0.8064, device='cuda:0')
evaluation_human_metric_hard_accuracy tensor(0.8023, device='cuda:0')
evaluation_human_metric_hard_f1 tensor(0.7640, device='cuda:0')
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8182, device='cuda:0')
evaluation_mouse_metric_hard_f1_1 tensor(0.7886, device='cuda:0')
evaluation_train_metric_pearson_r tensor(0., device='cuda:0')
evaluation_train_metric_spearman_r nan
evaluation_dev_metric_pearson_r tensor(-0.0865, device='cuda:0')
evaluation_dev_metric_spearman_r -0.1255236847234513
tensor(0.9040, device='cuda:0')
evaluation_train_mean_reciprocal_rank tensor(0.9040, device='cuda:0')
tensor(2.3482, device='cuda:0')
evaluation_dev_mean_reciprocal_rank tensor(2.3482, device='cuda:0')


align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6047, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7162, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5235, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6773, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4274, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5330, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4455, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.6075, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8636, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8763, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.7176, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6068, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7172, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5231, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6771, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4241, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5340, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4521, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.6121, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8594, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8738, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.7123, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6063, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7170, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5247, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6778, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4241, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5315, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4488, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.6089, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8619, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8765, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.7077, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6035, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7154, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5247, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6778, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4208, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5250, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4455, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.6038, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8586, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8741, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.7057, device='cuda:0')
a

...

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6191, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7237, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5350, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6826, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4323, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5209, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4356, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5870, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8454, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8652, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.6904, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6183, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7234, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5350, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6826, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4356, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5183, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4389, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5904, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8450, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8648, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.6913, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6228, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7256, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5375, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6838, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4323, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5169, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4356, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5850, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8417, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8625, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.6908, device='cuda:0')
a

align_evaluation_train_align_0.1_metric_hard_accuracy_align tensor(0.6261, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align tensor(0.7275, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_accuracy_align_mean tensor(0.5392, device='cuda:0')
align_evaluation_train_align_0.1_metric_hard_f1_align_mean tensor(0.6845, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align tensor(0.4307, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align tensor(0.5148, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_accuracy_align_mean tensor(0.4389, device='cuda:0')
align_evaluation_dev_align_0.1_metric_hard_f1_align_mean tensor(0.5874, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align tensor(0.8417, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_f1_align tensor(0.8624, device='cuda:0')
align_evaluation_train_align_0.2_metric_hard_accuracy_align_mean tensor(0.6908, device='cuda:0')
a

Stopped training at 30.875706214689266 epochs due to keyboard interrupt.


In [301]:
#the log of margin loss
#l.train(nEpochs)

------------------------------------------------

mouse:  tensor(0.1338, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1545, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8764, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8638, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0127, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.3512, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0306, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8485, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8340, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0146, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.2118, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1925, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8662, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8444, device='cuda:0', grad_fn=<Me

------------------------------------------------

mouse:  tensor(0.1504, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0945, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8754, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8651, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0103, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.3958, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1914, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8515, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8442, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0073, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.1339, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.2188, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8432, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8359, device='cuda:0', grad_fn=<Me

------------------------------------------------

mouse:  tensor(0.1053, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1473, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.9074, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8761, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0313, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.1062, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1373, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8385, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8251, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0133, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.1345, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1970, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8494, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8415, device='cuda:0', grad_fn=<Me

------------------------------------------------

mouse:  tensor(0.0924, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1038, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8381, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8308, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0073, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.1152, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0886, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8573, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8537, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0036, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.0868, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1863, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8286, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8271, device='cuda:0', grad_fn=<Me

align:  tensor(0.1113, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.2053, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8582, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8290, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0292, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

mouse:  tensor(0.1212, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1369, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8453, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8390, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0062, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.0575, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0616, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.8705, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.8661, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0044, device='cuda:0', grad

...

------------------------------------------------

mouse:  tensor(0.3217, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1842, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5822, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.4239, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1583, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.7412, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.2712, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.4834, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.3122, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1712, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.6165, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1682, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5691, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.5331, device='cuda:0', grad_fn=<Me

------------------------------------------------

mouse:  tensor(0.2624, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0735, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5814, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.5067, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0748, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.9368, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.3064, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5660, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.3743, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1918, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.6687, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.2230, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5087, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.4973, device='cuda:0', grad_fn=<Me

------------------------------------------------

mouse:  tensor(0.9842, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.4753, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.6052, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.4796, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1256, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

human:  tensor(0.8178, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0910, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.4939, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.3970, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0969, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.6027, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0711, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5567, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.5385, device='cuda:0', grad_fn=<Me

------------------------------------------------

human:  tensor(0.1635, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1998, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5123, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.3438, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1686, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

align:  tensor(0.5299, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.0508, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.5625, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.4891, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0733, device='cuda:0', grad_fn=<MeanBackward0>)
------------------------------------------------

mouse:  tensor(0.6547, device='cuda:0', grad_fn=<SelectBackward>) tensor(0.1989, device='cuda:0', grad_fn=<SelectBackward>)
-----------
 tensor(0.6625, device='cuda:0', grad_fn=<AddBackward0>) tensor(0.4792, device='cuda:0', grad_fn=<Me

metric_hard_accuracy_align: tensor(0.5017, device='cuda:0')
metric_hard_f1_align: tensor(0.0066, device='cuda:0')



In [304]:
rec_col.train

,align_cond_kl_loss,human_cond_kl_loss,loss,metric_hard_accuracy,metric_hard_f1,metric_pearson_r,metric_spearman_r,mouse_cond_kl_loss
0.0,0.100534,0.102254,0.305603,0.002302,0.004594,0.0,NaN,0.102815
1.0,0.090767,0.084271,0.262343,0.015054,0.029661,0.0,NaN,0.087305
2.0,0.080610,0.069721,0.224654,0.092712,0.169692,0.0,NaN,0.074322
3.0,0.070644,0.058283,0.193202,0.191092,0.320868,0.0,NaN,0.064274
4.0,0.061381,0.049707,0.167371,0.279908,0.437388,0.0,NaN,0.056283
5.0,0.053040,0.043285,0.146186,0.370495,0.540673,0.0,NaN,0.049861
6.0,0.045731,0.038542,0.128906,0.451961,0.622553,0.0,NaN,0.044633
7.0,0.039232,0.034683,0.114124,0.528823,0.691804,0.0,NaN,0.040210
8.0,0.033802,0.031621,0.101920,0.589569,0.741797,0.0,NaN,0.036498
9.0,0.029121,0.029098,0.091620,0.640308,0.780717,0.0,NaN,0.033401


In [305]:
rec_col.dev

,align_cond_kl_loss,human_cond_kl_loss,loss,metric_hard_accuracy,metric_hard_f1,metric_pearson_r,metric_spearman_r,mouse_cond_kl_loss
0.0,0.054680,NaN,0.054680,0.501650,0.009836,0.014805,0.009490,NaN
1.0,0.053776,NaN,0.053776,0.500825,0.009820,-0.002690,-0.009070,NaN
2.0,0.052965,NaN,0.052965,0.499175,0.009788,-0.018782,-0.025940,NaN
3.0,0.052466,NaN,0.052466,0.499175,0.009788,-0.039938,-0.050655,NaN
4.0,0.052349,NaN,0.052349,0.499175,0.019386,-0.065710,-0.081755,NaN
5.0,0.052311,NaN,0.052311,0.500825,0.028892,-0.086064,-0.106252,NaN
6.0,0.052412,NaN,0.052412,0.504125,0.050553,-0.099673,-0.124656,NaN
7.0,0.055740,NaN,0.055740,0.505776,0.065523,-0.107338,-0.131657,NaN
8.0,0.055736,NaN,0.055736,0.498350,0.064615,-0.111789,-0.140300,NaN
9.0,0.055832,NaN,0.055832,0.504125,0.082443,-0.115003,-0.154101,NaN
